# Interpretation of BertForSequenceClassification in captum

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import os

os.chdir('..')
os.listdir()

['.git',
 '.gitignore',
 '.idea',
 'config.py',
 'data',
 'document_processing.py',
 'main.py',
 'neuro.py',
 'notebooks',
 'temp',
 'test_model',
 'venv',
 'widgets.py',
 '__pycache__']

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
device

device(type='cpu')

In [3]:
from config import MODEL_PATH

# load model
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

In [4]:
def predict(inputs):
    return model(inputs)[0]

In [5]:
ref_token_id = tokenizer.pad_token_id  # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id  # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id  # A token used for prepending to the concatenated question-text word sequence

In [6]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(
        text,
        add_special_tokens=True,
        max_length=32,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    ).squeeze(0).tolist()
    # construct input token ids
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * (len(text_ids) - 2) + [sep_token_id]
    return torch.tensor([text_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)


def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)  # * -1
    return token_type_ids, ref_token_type_ids


def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids


def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [7]:
def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim=1)[0][0].unsqueeze(-1)

In [8]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

In [9]:
import json

# read classes.json
with open('data/classes.json', encoding='utf-8') as f:
    data = {key: value.split('/')[-1] for (key, value) in json.load(f).items()}

In [10]:
classes = list(set(list(data.values())))
classes.sort()

In [11]:
from document_processing import document2text, preprocess_text
import pandas as pd
import os

df = pd.DataFrame({'label': int(), 'text': str()}, index=[])
for key, value in data.items():
    df = df.append({'label': classes.index(value),
                    'text': document2text(os.path.join('data/docs', key))},
                   ignore_index=True)
    break
df['text'] = df['text'].apply(preprocess_text)
text = df['text'][0]
text

'Evaluation Only Created with Aspose Words Copyright 2003 2022 Aspose Pty Ltd ДОГОВОР г Москва 2012 г Общество с ограниченной ответственностью ООО именуемое в дальнейшем Поставщик в лице Генерального директора действующего на основании Устава с одной стороны и именуемое в дальнейшем Покупатель в лице Генерального директора действующего на основании именуемое в дальнейшем Покупатель с другой стороны вместе именуемые Стороны а индивидуально Сторона заключили настоящий договор поставки оборудования далее по тексту Договор о нижеследующем 1 Предмет договора 1 1 В соответствии с Договором Поставщик обязуется передать оборудование указанное в п HYPERLINK l p012 1 2 Договора далее по тексту Оборудование в собственность Покупателю а Покупатель обязуется принять и оплатить Оборудование в порядке и сроки указанные в Договоре 1 2 В Спецификации оборудования Приложение 1 к Договору являющейся неотъемлемой частью Договора Сторонами определены наименование Оборудования количество Оборудования стоимо

In [12]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

C:\Users\3wers\Anaconda3\envs\workspace\lib\site-packages\transformers\tokenization_utils_base.py:2342: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
model(input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3688,  0.3274,  0.3519, -0.4547,  0.1147]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [14]:
predict(input_ids)

tensor([[-0.3688,  0.3274,  0.3519, -0.4547,  0.1147]],
       grad_fn=<AddmmBackward>)

In [15]:
custom_forward(input_ids)

tensor([0.1316], grad_fn=<UnsqueezeBackward0>)

In [ ]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True)

In [16]:
score = predict(input_ids)

# print('Question: ', text)
label = torch.argmax(score[0]).cpu().detach().numpy()
label = classes[int(label)]
prob_ungrammatical = torch.softmax(score, dim=1)[0][0].cpu().detach().numpy()
prob = torch.softmax(score, dim=1)[0][1].cpu().detach().numpy()
print('Predicted Answer: ' + label + ', prob ungrammatical: ' + str(prob_ungrammatical))

Predicted Answer: Договоры оказания услуг, prob ungrammatical: 0.13150716


In [2]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
attributions_sum = summarize_attributions(attributions)

In [ ]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(
    attributions_sum,
    torch.softmax(score, dim=1)[0][0],
    torch.argmax(torch.softmax(score, dim=1)[0]),
    0,
    text,
    attributions_sum.sum(),
    all_tokens,
    delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])